# Multi-Cluster & Federation
Multi-cluster operation and federation are advanced features of Slurm. When multiple clusters are connected to the same slurmdb account services, multi-cluster operation allows job submission, monitoring, and configuration changes across clusters from the clients of any one of the clusters. On top of that, when a federation is configured, jobs submitted to a cluster member can be replicated to all or a subset of other clusters in the federation as sibling jobs, and a unified job ID space is used to uniquely identify a job globally. 

## Architecture
The architecture of this container lab is expanded for trying out the multi-cluster and federation features.  
![multi-cluster-and-federation](./multi-cluster-and-federation.drawio.svg)  

A new cluster, "lyoko", is added to the architecture and will use the existing slurmdbd services hosted on the containers slurm-lab-master-[1-2]. You can do this by uncommenting the corresponding objects in  `./compose.dev.yml` and updating the cluster.
```
podman compose -f compose.dev.yml up -d 
```
At this point, multi-cluster operation is enabled automatically, as they both use the same slurmdbd service. 

## Multi-Cluster Operation
Although this client container is only a part of the slurm-lab cluster, the slurm command can be used to query, control, and interact with the other cluster (lyoko) that uses the same slurmdbd service with the `-M` or `--clusters` option. 

In [ ]:
sinfo --long --clusters slurm-lab,lyoko
sinfo --N --long --clusters slurm-lab,lyoko

In [ ]:
squeue -la --clusters slurm-lab,lyoko

### Job submission
For job submission, you can list one or more clusters. One job will be submitted to the cluster that is estimated to start the job the earliest.

In [ ]:
# submit one job to slurm-lab, the local cluster
jobid_0=$( sbatch --ntasks=2 --parsable --time 00:10:00 endless-checksum-mpi.sh )

# submit one job to lyoko, the remote cluster
# parsable output format: <jobid>;<cluster>
IFS=';' read -r jobid_1 cluster_1 < <( sbatch --cluster lyoko --ntasks=2 --parsable --time 00:10:00 endless-checksum-mpi.sh )

# submit one job to multi cluster, see where it lands
# parsable output format: <jobid>;<cluster>
IFS=';' read -r jobid_2 cluster_2 < <( sbatch --cluster slurm-lab,lyoko --ntasks=2 --parsable --time 00:10:00 endless-checksum-mpi.sh )

cat <<EOF
Job ${jobid_0} submitted to local cluster (slurm-lab)
Job ${jobid_1} submitted to remote cluster (${cluster_1})
Job ${jobid_2} submitted to earliest available cluster ${cluster_2}
EOF

squeue -la --clusters slurm-lab,lyoko

In [ ]:
# cancel local job
scancel ${jobid_0}

# cancel job on specific cluster
scancel --clusters ${cluster_1} ${jobid_1}
scancel --clusters ${cluster_2} ${jobid_2}

# check queue
squeue -la --clusters slurm-lab,lyoko

IMPORTANT: If multiple clusters are given, `scancel` will cancel jobs with the specified job ID on ALL of them. Job IDs are NOT unique!!!

### scontrol
For `scontrol`, it is a little bit different. ONLY ONE cluster can be specified in the option; otherwise, a fatal error will occur. 

In [ ]:
# example of fatal error
scontrol --clusters slurm-lab,lyoko update nodename=ALL state=drain reason="this cmd is going to fail"

# draing nodes in the remote cluster 
scontrol --cluster lyoko update nodename=ALL state=drain reason="This would work"
sinfo --N --long --cluster lyoko

# undrain the node
scontrol --cluster lyoko update nodename=ALL state=resume

## Federation
Federation is another level of cluster integration on top of multi-cluster. When a federation of clusters is created, sibling jobs are created on all or some federation members whenever a new job is submitted, so the job can start earlier if resources are available on another cluster. 

In [ ]:
# Create Federation
sacctmgr show clusters
sacctmgr -i add federation world_without_danger
# two ways of adding cluster to federation
sacctmgr -i modify federation world_without_danger set clusters+=slurm-lab
sacctmgr -i modify cluster lyoko set federation=world_without_danger

# show federation
sacctmgr show federation

### Job Submission and Slurm commands
A job submitted to the federation has a globally unique job ID. The job's original cluster is distinguishable by the prefix of the job ID. By default, the slurm command shows information for the local cluster only. Use the `--federation` flag to show federation-related information.

In [ ]:
# submittion 3 jobs local, expecting the 3rd job will be executed on the remote cluster
jobid_0=$(sbatch --ntasks=2 --parsable --time 00:05:00 endless-checksum-mpi.sh)
jobid_1=$(sbatch --ntasks=2 --parsable --time 00:05:00 endless-checksum-mpi.sh)
jobid_2=$(sbatch --ntasks=2 --parsable --time 00:05:00 endless-checksum-mpi.sh)

# submit a job from cluster lyoko, expecting different prefix
jobid_3=$(ssh slurm-lab-master-lyoko sbatch --ntasks=2 --parsable --time 00:05:00 tutorials/endless-checksum-mpi.sh)

In [ ]:
set -x
# show global job queue
squeue -la --federation
squeue -la --federation -M slurm-lab,lyoko

# all available queue and nodes
sinfo --long --federation
sinfo --N --long --federation

set +x

Note that for the third job submitted locally (on slurm-lab), its sibling job is started on the remote cluster lyoko. The original job is in the "REVOKED" state and will not start on the original cluster. 

The job submitted via SSH on the cluster lyoko's master has a different job ID prefix, so job IDs are globally unique, and you can still tell which cluster a job is originated from.

In [ ]:
scancel ${jobid_0} ${jobid_1} ${jobid_2} ${jobid_3}
squeue -la --federation -M slurm-lab,lyoko

### Default to federation view
By setting [FederationParameters](/doc/slurm.conf.html#OPT_FederationParameters) in `slurm.conf`, the slurm commands will now default to showing the federated view. You don't have to set this variable on all the hosts, but it is always recommended to keep `slurm.conf` consistent across all the nodes in a cluster. 

In [ ]:
# add parameter to local slurm.conf only
ansible -m lineinfile -a 'path=/etc/slurm/slurm.conf regexp="^.*FederationParameters=.*$" line="FederationParameters=fed_display"' localhost

In [ ]:
# submittion 3 jobs local, expecting the 3rd job will be executed on the remote cluster
jobid_0=$(sbatch --ntasks=2 --parsable --time 00:05:00 endless-checksum-mpi.sh)
jobid_1=$(sbatch --ntasks=2 --parsable --time 00:05:00 endless-checksum-mpi.sh)
jobid_2=$(sbatch --ntasks=2 --parsable --time 00:05:00 endless-checksum-mpi.sh)

# submit a job from cluster lyoko, expecting different prefix
jobid_3=$(ssh slurm-lab-master-lyoko sbatch --ntasks=2 --parsable --time 00:05:00 tutorials/endless-checksum-mpi.sh)

In [ ]:
set -x
# show global job queue
squeue -la
squeue -la -M slurm-lab,lyoko

# all available queue and nodes
sinfo --long 
sinfo --N --long

set +x

In [ ]:
scancel ${jobid_0} ${jobid_1} ${jobid_2} ${jobid_3}
squeue -la -M slurm-lab,lyoko

In [ ]:
# revert the change
ansible -m lineinfile -a 'path=/etc/slurm/slurm.conf regexp="^.*FederationParameters=.*$" state="absent"' localhost

### Cluster state in federation
Federation member can have 4 different states
  - **ACTIVE**: Accepting job 
  - **INACTIVE**: Not running or accepting jobs
  - **DRAIN**: Not accepting new job. but job still running.
  - **DRAIN+REMOVE**: Set cluster to drain, and remove from federation once it is drained.

### Removing cluster from federation
Removing cluster lyoko and slurm-lab from federation world_without_danager by setting fedstate to "DRAIN+REMOVE"

In [ ]:
sacctmgr -i modify cluster lyoko set fedstate=DRAIN+REMOVE
# sacctmgr -i modify cluster slurm-lab set fedstate=DRAIN+REMOVE

#### Deleting federation
Deleting federation world_without_danager. It is better to drain all cluster before delete the federation.

In [ ]:
sacctmgr -i delete federation world_without_danger